In [1]:
import pandas as pd
import numpy as np
import nltk
df = pd.read_csv('../data/press_all.csv')

## Pre processing

In [2]:
# Replace nan with ""
df = df.fillna("")
df

,id,siren,url_article,title,text
0,0,572067684,http://www.autoactu.com/?page=2&id_actu=24240&...,autoactu.com - l'information automobile profes...,Bosch recrute des diplômés hautement qualifi...
1,1,790503973,http://www.ouest-france.fr/bretagne/lannion-22...,You have been blocked,Please enable JS and disable any ad blocker
2,2,433977980,http://www.businesswire.fr/news/fr/20160308006...,Axway s'associe à Navitas pour accompagner la ...,
3,3,495232373,http://www.next-finance.net/Sofimac-Partners-a...,Marchés Privés - Sofimac Partners apporte 450 ...,Présent au capital de cette jeune entreprise ...
4,4,572050169,http://rss.usinenouvelle.com/~r/a-la-une/~3/Gs...,"Les finances dans le vert, Latécoère veut mode...",\n\n\n\n \n\nIdentifiez-vous\n L'Usine Aéro P...
...,...,...,...,...,...
76509,76509,510050461,https://www.lejsl.com/edition-bresse/2019/10/1...,Sornay | Un salarié prend la succession de Lec...,JOURNAL DE SAÔNE ET LOIRE Conditions généra...
76510,76510,445720717,https://abonne.lunion.fr/id102153/article/2019...,,
76511,76511,480012277,https://www.lemondedudroit.fr/deals/66470-jean...,Jeantet conseille Formel D - LE MONDE DU DROIT...,UTILISATION DES COOKIES : en poursuivant votr...
76512,76512,414819409,http://www.leparisien.fr/val-de-marne-94/ile-d...,"Ile-de-France : à Villeneuve-Saint-Georges, le...","« S'ils veulent nous virer, qu'ils le fassent..."


In [3]:
#Delete all articles with length 0 
# Get names of articles with length 0
indexNames = df[ (df['text']) == "" ].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df

,id,siren,url_article,title,text
0,0,572067684,http://www.autoactu.com/?page=2&id_actu=24240&...,autoactu.com - l'information automobile profes...,Bosch recrute des diplômés hautement qualifi...
1,1,790503973,http://www.ouest-france.fr/bretagne/lannion-22...,You have been blocked,Please enable JS and disable any ad blocker
3,3,495232373,http://www.next-finance.net/Sofimac-Partners-a...,Marchés Privés - Sofimac Partners apporte 450 ...,Présent au capital de cette jeune entreprise ...
4,4,572050169,http://rss.usinenouvelle.com/~r/a-la-une/~3/Gs...,"Les finances dans le vert, Latécoère veut mode...",\n\n\n\n \n\nIdentifiez-vous\n L'Usine Aéro P...
5,5,794458075,http://feedproxy.google.com/~r/TracesEcrites/~...,La jeune pousse In’Air Solutions traque les po...,MENU Bourgogne-Franche-Comté Grand-Est - ...
...,...,...,...,...,...
76508,76508,493255848,http://www.actu-transport-logistique.fr//dossi...,,Transport Supply Chain Magazine | Dossier...
76509,76509,510050461,https://www.lejsl.com/edition-bresse/2019/10/1...,Sornay | Un salarié prend la succession de Lec...,JOURNAL DE SAÔNE ET LOIRE Conditions généra...
76511,76511,480012277,https://www.lemondedudroit.fr/deals/66470-jean...,Jeantet conseille Formel D - LE MONDE DU DROIT...,UTILISATION DES COOKIES : en poursuivant votr...
76512,76512,414819409,http://www.leparisien.fr/val-de-marne-94/ile-d...,"Ile-de-France : à Villeneuve-Saint-Georges, le...","« S'ils veulent nous virer, qu'ils le fassent..."


In [18]:
df.text[0]

"  Bosch recrute des diplômés hautement qualifiés en informatique mais très peu en France.  Bosch veut embaucher 14 000 jeunes diplômés en 2016. Pour près des deux tiers des postes qu’il compte créer, le groupe allemand cherche principalement des profils hautement qualifiés spécialisés dans l’informatique, des développeurs de logiciels pour des applications Web ou pour des systèmes de capteurs embarqués. Parmi ces embauches, 3 500 seront réalisées en Inde, 2 500 en Chine, 2 100 en Allemagne, et seulement 120 en France. L'entreprise emploie déjà plus de 15 000 développeurs de logiciels aujourd'hui. Elle emploie 375 000 personnes dans le monde, dont 132 000 en Allemagne, soit environ 17 600 personnes de plus qu’en 2014.Xavier Champagne \n\nInformations Légales\nConditions générales d'utilisation\nContacts\nDevenir annonceur\nVotre avis nous intéresse\n\n"

In [40]:
preprocessing(df.text[0])

['  ',
 'Bosch',
 'recrute',
 'un',
 'diplômé',
 'hautemer',
 'qualifié',
 'en',
 'informatique',
 'mais',
 'très',
 'peu',
 'en',
 'France',
 '.',
 ' ',
 'Bosch',
 'vouloir',
 'embaucher',
 '14',
 '000',
 'jeune',
 'diplômer',
 'en',
 '2016',
 '.',
 'pour',
 'près',
 'un',
 'deux',
 'tiers',
 'un',
 'poste',
 'qu’',
 'il',
 'compte',
 'créer',
 ',',
 'le',
 'groupe',
 'allemand',
 'cherche',
 'principalement',
 'un',
 'profil',
 'hautemer',
 'qualifié',
 'spécialisés',
 'dans',
 'l’',
 'informatique',
 ',',
 'un',
 'développeur',
 'de',
 'logiciel',
 'pour',
 'un',
 'application',
 'Web',
 'ou',
 'pour',
 'un',
 'système',
 'de',
 'capteur',
 'embarqué',
 '.',
 'parmi',
 'ce',
 'embauche',
 ',',
 '3',
 '500',
 'être',
 'réaliser',
 'en',
 'Inde',
 ',',
 '2',
 '500',
 'en',
 'Chine',
 ',',
 '2',
 '100',
 'en',
 'Allemagne',
 ',',
 'et',
 'seulement',
 '120',
 'en',
 'France',
 '.',
 'le',
 'entreprise',
 'emploie',
 'déjà',
 'plus',
 'de',
 '15',
 '000',
 'développeur',
 'de',
 'logici

In [39]:
import re
import contractions
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem.snowball import FrenchStemmer
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger

nlp = spacy.load('fr')
french_lemmatizer = LefffLemmatizer()
nlp.add_pipe(french_lemmatizer, name='lefff')

# Tokenize text
def preprocessing(text):
    # To lowercase 
    text = text.lower()
    # Remove numbers
    text = re.sub("\d+", "", text)
    # Delete punctuation 
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    text =  word_tokenize(text) 
    # Stemming words ex: books — book, looked — look
    # Lemmentization (same as stemming but based on knowledge base)
    result = []
    doc = nlp(df.text[0])
    for d in doc:
        result.append(d.lemma_)
        

    return result

def test(dataframe):
    dataframe['text'] = dataframe.apply(lambda row: preprocessing(row['text']), axis=1)

2020-03-23 10:48:01,379 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-03-23 10:48:01,381 - spacy_lefff.lefff - INFO - Token lefff_lemma already registered
2020-03-23 10:48:01,383 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-03-23 10:48:01,858 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer


In [25]:
stemmer= FrenchStemmer()

stemmer.stem('diplômés')

'diplôm'

In [89]:
%%time
import multiprocessing
chunks = np.array_split(df,4)
manager = multiprocessing.Manager()
threads = []
for index,chunk in enumerate(chunks):
    thread = multiprocessing.Process(target=test, args=(chunk,))
    thread.start()
    threads.append(thread)
for thread in threads:
    thread.join()
import pickle
with open('chunks.pkl', 'wb') as f:
    pickle.dump(chunks, f)
new_df = pd.concat(chunks)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
CPU times: user 146 ms, sys: 396 ms, total: 541 ms
Wall time: 2h 18min 12s


In [92]:
new_df

,id,siren,url_article,title,text
0,0,572067684,http://www.autoactu.com/?page=2&id_actu=24240&...,autoactu.com - l'information automobile profes...,Bosch recrute des diplômés hautement qualifi...
1,1,790503973,http://www.ouest-france.fr/bretagne/lannion-22...,You have been blocked,Please enable JS and disable any ad blocker
3,3,495232373,http://www.next-finance.net/Sofimac-Partners-a...,Marchés Privés - Sofimac Partners apporte 450 ...,Présent au capital de cette jeune entreprise ...
4,4,572050169,http://rss.usinenouvelle.com/~r/a-la-une/~3/Gs...,"Les finances dans le vert, Latécoère veut mode...",\n\n\n\n \n\nIdentifiez-vous\n L'Usine Aéro P...
5,5,794458075,http://feedproxy.google.com/~r/TracesEcrites/~...,La jeune pousse In’Air Solutions traque les po...,MENU Bourgogne-Franche-Comté Grand-Est - ...
...,...,...,...,...,...
76508,76508,493255848,http://www.actu-transport-logistique.fr//dossi...,,Transport Supply Chain Magazine | Dossier...
76509,76509,510050461,https://www.lejsl.com/edition-bresse/2019/10/1...,Sornay | Un salarié prend la succession de Lec...,JOURNAL DE SAÔNE ET LOIRE Conditions généra...
76511,76511,480012277,https://www.lemondedudroit.fr/deals/66470-jean...,Jeantet conseille Formel D - LE MONDE DU DROIT...,UTILISATION DES COOKIES : en poursuivant votr...
76512,76512,414819409,http://www.leparisien.fr/val-de-marne-94/ile-d...,"Ile-de-France : à Villeneuve-Saint-Georges, le...","« S'ils veulent nous virer, qu'ils le fassent..."


In [ ]:
# Check average length of article 
df['length'] = df.text.apply(lambda x: len(x))
max_ = df.length.max()
print("Max: {}".format(max_))
min_ = df.length.min()
print("Min: {}".format(min_))
mean = df.length.mean()
print("Mean: {}".format(mean))


In [14]:
f = open('data.txt', 'w')
for i,row in df.iterrows():
    f.write(row['text'] + '\n')  # python will convert \n to os.linesep
f.close()  

In [20]:
import fasttext
model = fasttext.train_unsupervised('../data/data.txt',thread=4)

In [21]:
model

In [24]:
model.get_nearest_neighbors('IPSEN')

AttributeError: '_FastText' object has no attribute 'get_nearest_neighbors'